#  第四章 智能体经典范式
## Motivation：为什么“重复造轮子”？
> 本章学习智能体经典范式，包括：ReAct、Plan-and-Solve、Reflection等。

目前市面过多高度抽象Agent框架，不利于理解智能体的运行原理。标准化逐渐不利于Agent的定制化。
🎯核心学习目标：
- 理解设计机制：理解ReAct、Plan-and-Solve、Reflection如何在代码层面实现。
- 暴露工程挑战：亲手处理那些被框架隐藏的难题，如模型输出解析、工具调用的重试、防止死循环等。
- 定制化能力：通过理解经典范式，能够定制化实现符合需求的智能体。

## Definition：
三种经典范式，定义了智能体组织“思考”与“行动”的不同方式：
ReAct（Reasoning and Acting）：
- 将“Reasoning”与“Acting”两个过程合并到一个循环中。
- 核心循环：不断重复Thought -> Action -> Observation 的循环。根据上一步的Observation动态调整下一步的Thought。
Plan-and-Solve：
- “三思而后行”，将任务明确解耦为两个阶段。
- 核心流程：
  1. 规划（Plan）：根据当前环境和任务，生成一个完整、清晰、分步骤的行动计划。
  2. 执行（Slove）：智能体严格按照这个（通用是静态的）计划执行。
Reflection：
- 赋予智能体“自我批判和修正”能力的范式。
- 核心循环：它引入了一个“事后（post-hoc）”的自我矫正循环：Execute -> Reflect -> Optimize。智能体先完成“初稿”后，进行自我结果的“评审”，根据“反馈”生成“修订稿”。

## Design：
### 实现基础：
环境依赖：`openai`、`python-dotenv`【包含第一章】
配置管理：`env.`文件存放`LLM_API_KEY`、`LLM_API_BASE`等配置项。
基础封装：`HelloAgentLLM`客户端类，封装OpenAI兼容接口的调用，默认用流式响应（`stream=True`）。
构建`llm_client`用于封装LLM模型，`Tools`用于封装工具函数与执行器。

而`ReAct`、`Plan-and-Solve`、`Reflection`则是不同的智能体组织方式，分别对应不同的思考与行动策略。

**拆解：`ReAct` **
该内容可以理解为一个状态机，维护当前**任务历史上下文（History）**，并且驱动一个**思考-行动-观察**。

工作流：
- 构建提示词：告诉LLM有什么工具、任务是什么、之前发生了什么，终止循环条件；
- 获取决策：LLM返回“思考”和“行动”
- 执行行动：代码解析LLM的意图，调用`ToolExecutor`
- 获取反馈：工具返回结果（Observation）
- 闭环：将结果存入历史，进入下一轮

1. 提示模版`REACT_PROMPT_TEMPLATE`
- 工具注入{tools}，在`ReActAgent`中，`tools_desc`调用`tools.py`中`getAvailableTools`获取Tools与信息
- 格式约束：要求`Thought`与`Action`便于后续做**正则解析** 
- 动态历史{history}：LLM填入短期记忆

2. 循环`run`的方法
ReAct的执行：
- 循环控制：`while current_step < self.max_steps`，防止因检索问题陷入死循环
- LLM调用：`self.llm_client.think`发送保护工具描述和历史记录的完整Prompt（此处制作了`user`的）
- 终止判断：
```python
if action.startswith("Finish"):
    return final_answer
```

> 为什么通过这类型的PROMPT就可以形成T-A-F的过程？且模型会按步执行。

- 历史更新：将“行动”和“结果”写入“记忆”，供下一轮LLM参考
```python
self.history.append(f"Action: {action}")
self.history.append(f"Observation: {observation}")
```

3. 输出解析与工具执行（`_parse_*`方法）
连接LLM文本输出与Python代码执行的处理。
- 正则解析（`re.search`，`re.match`）：
  - LLM输出为纯文本
  - `_parse_output`负责把`Thought`与`Action`分离
  - `_parse_action`负责把`Search`工具名和`华为手机`(参数分离)
- 工具调用：使用`ToolExexutor`解析出名字找到函数，并且执行
```python
tool_function = self.tool_executor.getTool(tool_name)
observation = tool_function(tool_input)
```

应用执行的流程：
- 查询手机Q传入
- LLM发现自己不知道实时信息（知识截止）
- Thought：我需要搜索最新的手机 ->Action：`Search[华为最新手机]`
- Observation：（工具返回搜索结果）
- LLM看到Observation
- Thought：我找了信息，可以回答 ->Action:`Finish[华为手机是...]`
